In [10]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from tensorboardX import SummaryWriter      

import torchvision
import torchvision.transforms as transforms

from models import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 256
model_name = "VGG16_quant_unstructured_pruning"
model = VGG16_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [11]:
PATH = "result/VGG16_quant_mixed_pruning/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9041/10000 (90%)



In [12]:
# C2F (Coarse-to-Fine) Pruning Implementation
# Stages: Filter-wise -> Channel-wise -> Unstructured

import torch.nn.utils.prune as prune

def c2f_pruning(model, target_sparsity=0.8):
    """
    Applies Coarse-to-Fine (C2F) pruning:
    1. Filter-wise pruning (Structured, dim=0) - Removes entire filters
    2. Channel-wise pruning (Structured, dim=1) - Removes entire input channels
    3. Unstructured pruning (Fine-grained) - Removes individual weights
    """
    print(f"Applying C2F Pruning with target sparsity {target_sparsity}...")
    
    # Define sparsity budgets for coarse stages
    # These values can be tuned based on sensitivity analysis
    filter_sparsity = 0.3  # 30% of filters (Coarse)
    channel_sparsity = 0.3 # 30% of channels (Coarse)
    
    for name, module in model.named_modules():
        if isinstance(module, QuantConv2d):
            print(f"Pruning {name}...")
            
            # Stage 1: Filter-wise pruning (dim=0)
            # Removes output channels (filters) based on L1 norm
            prune.ln_structured(module, name='weight', amount=filter_sparsity, n=1, dim=0)
            
            # Stage 2: Channel-wise pruning (dim=1)
            # Removes input channels based on L1 norm
            prune.ln_structured(module, name='weight', amount=channel_sparsity, n=1, dim=1)
            
            # Calculate current sparsity after structured steps
            total_params = module.weight.nelement()
            zeros = torch.sum(module.weight == 0).item()
            current_sparsity = zeros / total_params
            
            # Stage 3: Unstructured pruning to reach target
            if target_sparsity > current_sparsity:
                # Calculate required unstructured sparsity to reach total target
                # Formula: target = current + (1 - current) * u
                # => u = (target - current) / (1 - current)
                unstructured_amount = (target_sparsity - current_sparsity) / (1 - current_sparsity)
                
                # Clip to valid range [0, 1] to avoid errors if target is already met
                unstructured_amount = max(0.0, min(1.0, unstructured_amount))
                
                print(f"  Structured sparsity: {current_sparsity:.4f}. Applying unstructured amount: {unstructured_amount:.4f}")
                prune.l1_unstructured(module, name='weight', amount=unstructured_amount)
            
            # Final check
            zeros = torch.sum(module.weight == 0).item()
            print(f"  Final Sparsity: {zeros/total_params:.4f}")

# Apply C2F pruning
c2f_pruning(model, target_sparsity=0.8)

Applying C2F Pruning with target sparsity 0.8...
Pruning features.0...
  Structured sparsity: 0.5312. Applying unstructured amount: 0.5733
  Final Sparsity: 0.7998
Pruning features.3...
  Structured sparsity: 0.5056. Applying unstructured amount: 0.5955
  Final Sparsity: 0.8000
Pruning features.7...
  Structured sparsity: 0.5056. Applying unstructured amount: 0.5955
  Final Sparsity: 0.8000
Pruning features.10...
  Structured sparsity: 0.5056. Applying unstructured amount: 0.5955
  Final Sparsity: 0.8000
Pruning features.14...
  Structured sparsity: 0.5084. Applying unstructured amount: 0.5932
  Final Sparsity: 0.8000
Pruning features.17...
  Structured sparsity: 0.5111. Applying unstructured amount: 0.5909
  Final Sparsity: 0.8000
Pruning features.20...
  Structured sparsity: 0.5111. Applying unstructured amount: 0.5909
  Final Sparsity: 0.8000
Pruning features.24...
  Structured sparsity: 0.5111. Applying unstructured amount: 0.5909
  Final Sparsity: 0.8000
Pruning features.27...
  S

In [ ]:
# Apply mixed pruning with 80% target sparsity


Applying mixed pruning with target amount=0.8...
Global sparsity achieved: 80.01%


In [ ]:
print(list(model.features[40].named_parameters())) # check whether there is mask, weight_org, ...
print(model.features[40].weight) # check whether there are many zeros

In [14]:
### Check sparsity ###
mask1 = model.features[40].weight_mask
sparsity_mask1 = (mask1 == 0).sum() / mask1.nelement()

print("Sparsity level: ", sparsity_mask1)

Sparsity level:  tensor(0.8000, device='cuda:0')


In [8]:
## check accuracy after pruning

model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 1000/10000 (10%)



In [15]:
## Start finetuning (training here), and see how much you can recover your accuracy ##
## You can change hyper parameters such as epochs or lr ##
lr = 0.0005
weight_decay = 1e-5
epochs = 300
best_prec = 0
#momentum = 0.9

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)


Epoch: [0][0/196]	Time 4.467 (4.467)	Data 4.441 (4.441)	Loss 9.4826 (9.4826)	Prec 11.719% (11.719%)
Epoch: [0][100/196]	Time 0.030 (0.082)	Data 0.001 (0.045)	Loss 1.3223 (2.0965)	Prec 49.219% (30.817%)
Validation starts
Test: [0/40]	Time 3.839 (3.839)	Loss 0.6820 (0.6820)	Prec 76.172% (76.172%)
 * Prec 74.140% 
best acc: 74.140000
Epoch: [1][0/196]	Time 4.788 (4.788)	Data 4.768 (4.768)	Loss 0.7150 (0.7150)	Prec 78.516% (78.516%)
Epoch: [1][100/196]	Time 0.049 (0.089)	Data 0.001 (0.048)	Loss 0.5214 (0.6287)	Prec 81.641% (78.353%)
Validation starts
Test: [0/40]	Time 3.988 (3.988)	Loss 0.5466 (0.5466)	Prec 80.078% (80.078%)
 * Prec 80.890% 
best acc: 80.890000
Epoch: [2][0/196]	Time 4.609 (4.609)	Data 4.591 (4.591)	Loss 0.4364 (0.4364)	Prec 86.328% (86.328%)
Epoch: [2][100/196]	Time 0.031 (0.070)	Data 0.001 (0.046)	Loss 0.4420 (0.4528)	Prec 84.375% (84.081%)
Validation starts
Test: [0/40]	Time 3.809 (3.809)	Loss 0.4965 (0.4965)	Prec 81.250% (81.250%)
 * Prec 82.260% 
best acc: 82.260000
E

In [16]:
## check your accuracy again after finetuning
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9022/10000 (90%)



In [17]:
from models import QuantConv2d

#### check global sparsity for weight_int is near 80% #####
# Iterate through all QuantConv2d layers and compute weight_int sparsity

w_bit = 4
total_zeros = 0
total_elements = 0

for name, module in model.named_modules():
    if isinstance(module, QuantConv2d):
        weight_q = module.weight_q
        w_alpha = module.weight_quant.wgt_alpha
        w_delta = w_alpha / (2**(w_bit-1) - 1)
        
        weight_int = weight_q / w_delta
        
        zeros = (weight_int == 0).sum().item()
        elements = weight_int.nelement()
        
        total_zeros += zeros
        total_elements += elements

global_sparsity = total_zeros / total_elements
print(f"Global sparsity for weight_int: {global_sparsity:.4f} ({global_sparsity*100:.2f}%)")

Global sparsity for weight_int: 0.8792 (87.92%)


Sparsity level:  tensor(0.8387, device='cuda:0')
